In [1]:
# add rxns making products of interest
import pandas as pd
import cobra
from copy import deepcopy

import sys
import os,shutil
sys.path.append('../pycore/')
from utils import extract_details_from_rxnid
from gsm_custom_functions import *


In [2]:
df_rxns = pd.read_excel('./input/pathways.xlsx')
copy_input_files = True # set to False if you want to customize them
# Manually create input/phenotype_common.txt

In [3]:
model = cobra.io.load_json_model('../build_model/input/iRhtoC.json')
model = deepcopy(model)
model.solver = 'cplex'
newmodel_path = './input/iRhtoC_pathwayadd.json'

In [4]:
path_gams = '../GAMS/'
path_app_gams = '../application/input/GAMS_model_application/'
# copy files from GAMS/model folder into path_app_gams, overwriting existing files
if copy_input_files:
	for f in os.listdir(path_gams+'model/'):
		shutil.copy(path_gams+'model/'+f, path_app_gams)

In [5]:
# model media - same as scRBA
model.objective = dict()

model.reactions.ATPM_c.lower_bound = 1
model.reactions.EX_glc__D_e.bounds = (-13.21,1000)
model.reactions.BIOMASS_RT_CLIM.bounds = (0.1, 0.1)
model.reactions.EX_o2_e.bounds = (-1000,1000)
model.reactions.EX_co2_e.bounds = (-1000,1000)

# Allow YNB media uptake
model.reactions.EX_thm_e.bounds = (-1000,1000)
model.reactions.EX_ribflv_e.bounds = (-1000,1000)
model.reactions.EX_nac_e.bounds = (-1000,1000)
model.reactions.EX_pydxn_e.bounds = (-1000,1000)
model.reactions.EX_fol_e.bounds = (-1000,1000)
model.reactions.EX_pnto__R_e.bounds = (-1000,1000)
model.reactions.EX_4abz_e.bounds = (-1000,1000)
model.reactions.EX_inost_e.bounds = (-1000,1000)

In [6]:
prod_rxns = dict()
prods = set()

sij_add = set()
added_rxns = set()

# add rxns to both models
for i in df_rxns.index:
    rxnid = extract_details_from_rxnid(df_rxns.id[i])[1]
    eqn = df_rxns.reaction[i]
    eqn = eqn.replace('MET-', '')
    p = df_rxns.loc[i, 'product']
    
    rxn = cobra.Reaction(rxnid)
    model.add_reactions([rxn])
    rxn.reaction = eqn
    rxn.bounds = (0,0)

    # add to RBA model
    met_dict = metabolites_dict_from_reaction_equation_RBA(rxn.reaction)
    for k,v in met_dict.items():
        if v != '' and k != '':
            sij_add.add("'" + k + "'.'" + df_rxns.id[i] + "' " + str(v))
    
    ## Example of how you might count 2 related products together
    if p == '3hpp':
        prod_rxns['3hppa'].append(rxnid)
        prod_rxns['3hppb'].append(rxnid)
    prods.add(p)
    if p in prod_rxns.keys():
        prod_rxns[p].append(rxnid)
    else:
        prod_rxns[p] = [rxnid]
    added_rxns.add(df_rxns.id[i])
prods = list(prods)
print(prods)

unknown metabolite 'hxdcal_c' created
unknown metabolite 'hxdcol_c' created
unknown metabolite '2etmal_m' created
unknown metabolite '2etmal_c' created
unknown metabolite '3etmal_c' created
unknown metabolite '2oxptn_c' created
unknown metabolite 'btal_c' created
unknown metabolite 'btoh_c' created
unknown metabolite 'citm_m' created
unknown metabolite 'citac_m' created
unknown metabolite '3mmal_m' created
unknown metabolite 'btoh_e' created
unknown metabolite 'tag_e' created
unknown metabolite 'cou_c' created
unknown metabolite 'nrgnch_c' created
unknown metabolite 'nrgn_c' created
unknown metabolite 'nrgn_e' created
unknown metabolite 'cou_e' created
unknown metabolite 'citm_c' created
unknown metabolite 'citm_e' created
unknown metabolite '3hbcoa__R_c' created
unknown metabolite 'polyhb_c' created
unknown metabolite 'polyhb_e' created
unknown metabolite 'amorph_c' created
unknown metabolite 'nhemrd_c' created
unknown metabolite 'arte_c' created
unknown metabolite 'nhemox_c' created


In [ ]:
# update sij
def process_GAMS_file(f):
    with open(f, 'r') as f:
        lines = f.readlines()
    # remove all lines that are empty or only contain '/'
    lines = [x.replace('\n','') for x in lines if x.strip() != '/' and x.strip() != '']
    return lines
sij = process_GAMS_file('./input/GAMS_model_application/RBA_sij.txt')
# add new lines
sij += list(sij_add)
# remove duplicates
sij = list(set(sij))
with open('./input/GAMS_model_application/RBA_sij.txt', 'w') as f:
    f.write('\n'.join(['/'] + sij + ['/']))

# add rxns
with open('./input/GAMS_model_application/RBA_rxns_add.txt', 'w') as f:
    f.write('\n'.join(['/'] + list(added_rxns) + ['/']))
allrxns = process_GAMS_file('./input/GAMS_model_application/RBA_rxns.txt')
allrxns += ["'" + x + "'" for x in added_rxns]
allrxns = list(set(allrxns))
with open('./input/GAMS_model_application/RBA_rxns.txt', 'w') as f:
    f.write('\n'.join(['/'] + allrxns + ['/']))

# add prods
species = process_GAMS_file('./input/GAMS_model_application/RBA_species.txt')
# add 'MET-' to products
species += ["'MET-" + x + "'" for x in prods]
species = list(set(species))
with open('./input/GAMS_model_application/RBA_species.txt', 'w') as f:
    f.write('\n'.join(['/'] + species + ['/']))

In [ ]:
cobra.io.save_json_model(model, newmodel_path)

 # Run FBA

In [ ]:
fba_yields = {p:0 for p in prods}

for p in prods:
    if p in ['3hppa', '3hppb']:
        exrxn = 'EX_3hpp_e'
    else:
        exrxn = 'EX_' + p + '_e'
    if p in ['citm', 'polyhb']:
        model.reactions.THRA_c.knock_out()
    for r in prod_rxns[p]:
        model.reactions.get_by_id(r).bounds = (0,1000)
    model.reactions.get_by_id(exrxn).objective_coefficient = 1
    fba = model.optimize()
    
    for r in prod_rxns[p]:
        model.reactions.get_by_id(r).bounds = (0,0)
    if p in ['citm', 'polyhb']:
        model.reactions.THRA_c.bounds = (0,1000)
    model.objective = dict()
    
    fba_yields[p] = fba[exrxn]
    #print(p, fba.status, fba[exrxn], fba['EX_glc__D_e'])
    print('product exchange flux: '+str(fba[exrxn]))
    print('product exchange flux / glc uptake:',str(fba[exrxn]/fba['EX_glc__D_e']))

 # Create RBA files from template

In [ ]:
# copy runRBA_max_prod.gms
# if copy_input_files:
# 	for p in prods:
		# copy 

 # Run RBA

In [ ]:
# shutil.copy(os.path.join(path_gams, 'runRBA_max_prod.gms'),
#             os.path.join(path_out, 'runRBA_max_prod.gms'));
# shutil.copy(os.path.join(path_gams, 'cplex.opt'),
#             os.path.join(path_out, 'cplex.opt'));

for p in prods:
        cmds = ['cd ' + p,
                'module load gams',
                'python3 runRBA.py']
        os.system('\n'.join(cmds))

 # Extract RBA yields

In [ ]:
RBA_result_dict = dict()
for p in prods:
    with open('./output_max/' + p + '/report.txt') as f:
        text = f.read().split('\n')

    for t in text:
        k,v = t.split('\t')
        try:
            RBA_result_dict[p][k] = float(v)
        except:
            RBA_result_dict[p][k] = v
            
    print(RBA_result_dict[p]['vprod'])
    #print(resdict['vglc'])

In [ ]:
# copy compiled_results_template.xlsx to compiled_results.xlsx
shutil.copy(os.path.join('./compiled_results_template.xlsx'),
			os.path.join('./compiled_results.xlsx'))
# write FBA yields to rateFBA column (2nd how has headers)
df = pd.read_excel('./compiled_results.xlsx', header=1)
for p in prods:
	df.loc[df['product'] == p, 'rateFBA'] = fba_yields[p]
	df.loc[df['product'] == p, 'rateRBA'] = RBA_result_dict[p]['vprod']
df.to_excel('./compiled_results.xlsx', index=False)
